
# Workforce Optimization — MIP (PuLP) + QUBO (neal) on One Dataset (Colab-Ready)

This notebook assigns employees to shift "seats" to meet staffing needs while honoring availability, skills, and preferences.

We use one **synthetic dataset** for both approaches:

1. **Classical MIP (Mixed-Integer Programming)**
   - Binary assignment \(x_{e,s,p}\): employee \(e\) fills seat \(p\) on shift \(s\).
   - **Hard constraints:** seat coverage, skill compatibility, availability, at most one seat per day per employee, max weekly seats.
   - **Objective:** minimize wage cost **minus** preference benefit.

2. **QUBO (time/seat-indexed)**
   - Variables \(x_{e,s,p}\) (only for eligible combos).
   - **Penalties:** each seat filled exactly once, at most one seat per employee per day, (skills/availability handled by excluding variables).
   - **Objective:** wage cost minus preference benefit.
   - Solved via **simulated annealing** (`neal`).

We then compare solutions and print coverage & cost summaries.


In [ ]:

# Install dependencies if missing
def _silent_imports():
    flags = {"pulp": False, "dimod": False, "neal": False}
    try:
        import pulp
        flags["pulp"] = True
    except Exception:
        pass
    try:
        import dimod
        flags["dimod"] = True
    except Exception:
        pass
    try:
        import neal
        flags["neal"] = True
    except Exception:
        pass
    return flags

flags = _silent_imports()
if not flags["pulp"]:
    %pip -q install pulp
if not flags["dimod"] or not flags["neal"]:
    %pip -q install dimod neal

flags = _silent_imports()
print("PuLP:", flags["pulp"], "| dimod:", flags["dimod"], "| neal:", flags["neal"])


In [ ]:

# ==== One Synthetic Dataset ====
import numpy as np, pandas as pd

rng = np.random.default_rng(101)

# Parameters
E = 24                     # employees
DAYS = 5                   # days in horizon
SHIFTS_PER_DAY = 2         # per day (e.g., Morning/Evening)
S = DAYS * SHIFTS_PER_DAY  # number of shifts
SKILLS = ["A","B"]         # two skill types
SEATS_PER_SHIFT = 3        # seats per shift
MAX_SEATS_PER_EMP = 4      # weekly max for each employee

employees = [f"E{i+1}" for i in range(E)]
days = [f"Day{d+1}" for d in range(DAYS)]
shift_names = [f"{days[d]}_{'M' if k==0 else 'E'}" for d in range(DAYS) for k in range(SHIFTS_PER_DAY)]
seats = list(range(SEATS_PER_SHIFT))

# Employee attributes
skill_primary = rng.choice(SKILLS, size=E, p=[0.55, 0.45])
# wage per shift (e.g., hours * hourly_rate); keep simple
wage = rng.integers(60, 121, size=E)  # $ per shift
# availability: Bernoulli (0.8)
availability = (rng.random((E, S)) < 0.8).astype(int)
# preferences: [0,1], independent per (e,s); higher is better
preferences = rng.random((E, S))

# Shift seat skill requirements (each seat has a skill label; first seat A, second B, third random)
seat_skill = {}
for s in range(S):
    seat_skill[(s,0)] = "A"
    seat_skill[(s,1)] = "B"
    seat_skill[(s,2)] = rng.choice(SKILLS)

# Build DataFrames
df_employees = pd.DataFrame({
    "emp": employees,
    "skill": skill_primary,
    "wage": wage
})
df_shifts = pd.DataFrame({
    "shift_id": list(range(S)),
    "shift_name": shift_names,
    "day": [i//SHIFTS_PER_DAY for i in range(S)]
})
# eligibility matrix: employee e eligible for seat (s,p) if available and skill matches seat_skill
eligible = {}
for ei in range(E):
    for s in range(S):
        for p in seats:
            if availability[ei, s] == 1 and skill_primary[ei] == seat_skill[(s,p)]:
                eligible[(ei,s,p)] = 1
            else:
                eligible[(ei,s,p)] = 0

# preference benefit scaled to dollars (benefit per shift)
PREF_BENEFIT = 30.0  # scale factor for preferences in dollars
pref_benefit = preferences * PREF_BENEFIT

print("Employees:", E, "| Shifts:", S, "| Seats per shift:", SEATS_PER_SHIFT)
df_employees.head(), df_shifts.head()



## Part 1 — Classical MIP (PuLP) with Heuristic Fallback

**Decision:** \(x_{e,s,p}\in\{0,1\}\) for eligible (employee, shift, seat).  
**Constraints:**
- **Seat coverage:** \(\sum_e x_{e,s,p} = 1\) for all seats \((s,p)\).
- **One seat per day per employee:** \(\sum_{s \in \text{day}(d)}\sum_p x_{e,s,p} \le 1\).
- **Weekly max seats per employee:** \(\sum_{s,p} x_{e,s,p} \le \text{Max}_e\).
- **Eligibility:** variables only created for eligible combos.

**Objective:** minimize \(\sum_{e,s,p} \big(\text{wage}_e - \text{prefBenefit}_{e,s}\big) x_{e,s,p}\).


In [ ]:

import numpy as np, pandas as pd

try:
    import pulp
    HAVE_PULP = True
except Exception:
    HAVE_PULP = False

def solve_mip_workforce(E, S, seats, df_employees, df_shifts, eligible, pref_benefit, MAX_SEATS_PER_EMP):
    prob = pulp.LpProblem("WorkforceAssignment", pulp.LpMinimize)
    # Decision variables only for eligible combos
    x = {}
    for ei in range(E):
        for s in range(S):
            for p in seats:
                if eligible[(ei,s,p)] == 1:
                    x[(ei,s,p)] = pulp.LpVariable(f"x_{ei}_{s}_{p}", lowBound=0, upBound=1, cat="Binary")

    # Objective
    for (ei,s,p), var in x.items():
        w = df_employees.loc[ei, "wage"]
        benefit = pref_benefit[ei, s]
        prob += (w - benefit) * var
    prob.setObjective(prob.objective)  # ensure set

    # Seat coverage
    for s in range(S):
        for p in seats:
            prob += pulp.lpSum(x[(ei,s,p)] for ei in range(E) if (ei,s,p) in x) == 1, f"cover_{s}_{p}"

    # One seat per day per employee
    for ei in range(E):
        for d in range(df_shifts["day"].max()+1):
            shifts_d = df_shifts[df_shifts["day"]==d]["shift_id"].tolist()
            prob += pulp.lpSum(x[(ei,s,p)] for s in shifts_d for p in seats if (ei,s,p) in x) <= 1, f"oneday_{ei}_{d}"

    # Weekly max seats per employee
    for ei in range(E):
        prob += pulp.lpSum(x[(ei,s,p)] for s in range(S) for p in seats if (ei,s,p) in x) <= MAX_SEATS_PER_EMP, f"maxweek_{ei}"

    _ = prob.solve(pulp.PULP_CBC_CMD(msg=False))
    status = pulp.LpStatus[prob.status]
    if status not in ("Optimal", "Feasible"):
        return status, None, None, None

    x_sol = {k: int(round(pulp.value(var) or 0)) for k,var in x.items()}
    obj = float(pulp.value(prob.objective))
    return status, x_sol, obj, x

def greedy_fallback(E, S, seats, df_employees, df_shifts, eligible, pref_benefit, MAX_SEATS_PER_EMP):
    # Greedy: iterate seats; pick eligible employee with smallest (wage - benefit)
    assigned = {}
    # Track constraints
    one_per_day = {(ei,d):0 for ei in range(E) for d in range(df_shifts["day"].max()+1)}
    week_count = {ei:0 for ei in range(E)}
    obj = 0.0

    for s in range(S):
        for p in seats:
            cand = []
            d = int(df_shifts.loc[s, "day"])
            for ei in range(E):
                if eligible[(ei,s,p)] != 1:
                    continue
                if one_per_day[(ei,d)] >= 1:
                    continue
                if week_count[ei] >= MAX_SEATS_PER_EMP:
                    continue
                w = df_employees.loc[ei,"wage"]
                benefit = pref_benefit[ei, s]
                score = (w - benefit)
                cand.append((score, ei))
            if not cand:
                # allow breaking one-per-day as last resort
                for ei in range(E):
                    if eligible[(ei,s,p)] != 1: continue
                    if week_count[ei] >= MAX_SEATS_PER_EMP: continue
                    w = df_employees.loc[ei,"wage"]; benefit = pref_benefit[ei, s]
                    cand.append((w-benefit, ei))
            cand.sort(key=lambda t: t[0])
            score, chosen = cand[0]
            assigned[(chosen,s,p)] = 1
            obj += score
            d = int(df_shifts.loc[s, "day"])
            one_per_day[(chosen,d)] += 1
            week_count[chosen] += 1
    return "Heuristic", assigned, obj, None

if HAVE_PULP:
    status_mip, x_sol_mip, obj_mip, varmap = solve_mip_workforce(
        E, S, seats, df_employees, df_shifts, eligible, pref_benefit, MAX_SEATS_PER_EMP
    )
else:
    status_mip, x_sol_mip, obj_mip, varmap = greedy_fallback(
        E, S, seats, df_employees, df_shifts, eligible, pref_benefit, MAX_SEATS_PER_EMP
    )

print("MIP/Greedy status:", status_mip, "| Objective:", obj_mip)


In [ ]:

# Summaries for MIP/Greedy
import pandas as pd, numpy as np

assign_rows = []
for (ei,s,p), val in x_sol_mip.items():
    if val == 1:
        assign_rows.append({"emp": employees[ei], "shift": int(s), "seat": int(p), "day": int(df_shifts.loc[s,"day"])})
df_assign = pd.DataFrame(assign_rows).sort_values(["day","shift","seat"])

# Coverage check
coverage = df_assign.groupby(["shift","seat"]).size().reset_index(name="filled")
unfilled = coverage[coverage["filled"]!=1]
print("Unfilled seats (should be 0 rows):")
display(unfilled.head())

# Employee loads
emp_load = df_assign.groupby("emp").size().reset_index(name="seats_assigned")
emp_cost = df_assign.merge(df_employees, on="emp", how="left").groupby("emp")["wage"].sum().reset_index(name="total_wage")
df_emp_summary = emp_load.merge(emp_cost, on="emp", how="left").fillna(0).sort_values("seats_assigned", ascending=False)

display(df_assign.head())
display(df_emp_summary.head())



## Part 2 — QUBO (Seat-Indexed) with `neal`

We create binary variables \(x_{e,s,p}\) **only for eligible** combos.

**Penalties & Objective:**
- **Seat filled exactly once:** \((1 - \sum_e x_{e,s,p})^2\)
- **At most one seat per employee per day:** pairwise penalty on any two assigned seats in the same day for the same employee.
- **Cost/Preference objective:** add linear term \((\text{wage}_e - \text{prefBenefit}_{e,s}) x_{e,s,p}\).

This yields a **QUBO** minimized by simulated annealing.


In [ ]:

from collections import defaultdict
import numpy as np, dimod, neal

# Build variable index only for eligible combos
var_index = {}
rev_index = {}
vid = 0
for ei in range(E):
    for s in range(S):
        for p in seats:
            if eligible[(ei,s,p)] == 1:
                var_index[(ei,s,p)] = vid
                rev_index[vid] = (ei,s,p)
                vid += 1

# Penalty weights
A = 8.0   # seat coverage
B = 6.0   # one-per-day per employee
Q = defaultdict(float)

# (1) Seat coverage: (1 - sum_e x)^2
for s in range(S):
    for p in seats:
        vars_sp = [var_index[(ei,s,p)] for ei in range(E) if (ei,s,p) in var_index]
        # linear -A for each var
        for v in vars_sp:
            Q[(v,v)] += -A
        # pairwise +2A for distinct pairs
        for i_idx in range(len(vars_sp)):
            for j_idx in range(i_idx+1, len(vars_sp)):
                Q[(vars_sp[i_idx], vars_sp[j_idx])] += 2*A

# (2) One seat per day per employee: for fixed (ei,d), penalize pairwise assignments in same day
for ei in range(E):
    for d in range(df_shifts["day"].max()+1):
        shifts_d = df_shifts[df_shifts["day"]==d]["shift_id"].tolist()
        vars_ed = [var_index[(ei,s,p)] for s in shifts_d for p in seats if (ei,s,p) in var_index]
        for a_idx in range(len(vars_ed)):
            for b_idx in range(a_idx+1, len(vars_ed)):
                Q[(vars_ed[a_idx], vars_ed[b_idx])] += B  # penalize having both

# (3) Objective linear terms
for (ei,s,p), v in var_index.items():
    w = df_employees.loc[ei,"wage"]
    benefit = pref_benefit[ei, s]
    Q[(v,v)] += (w - benefit)

# Build and solve
bqm = dimod.BinaryQuadraticModel.from_qubo(dict(Q))
sampleset = neal.SimulatedAnnealingSampler().sample(bqm, num_reads=1500)
best = sampleset.first
x_qubo = best.sample  # dict vid -> 0/1

# Decode assignment
assign_rows_q = []
for v, val in x_qubo.items():
    if val == 1:
        ei,s,p = rev_index[v]
        assign_rows_q.append({"emp": employees[ei], "shift": int(s), "seat": int(p), "day": int(df_shifts.loc[s,"day"])})
df_assign_q = pd.DataFrame(assign_rows_q).sort_values(["day","shift","seat"])

# Summary
cov_q = df_assign_q.groupby(["shift","seat"]).size().reset_index(name="filled")
unfilled_q = cov_q[cov_q["filled"]!=1]
print("QUBO — Unfilled seats (should be 0 rows ideally):")
display(unfilled_q.head())

# Cost
def total_cost(rows):
    if rows.empty: return 0.0
    merged = rows.merge(df_employees, on="emp", how="left")
    # preference benefit by (emp, shift) index
    benefit = 0.0
    for _,r in rows.iterrows():
        ei = employees.index(r["emp"]); s = int(r["shift"])
        benefit += float(pref_benefit[ei, s])
    return float(merged["wage"].sum() - benefit)

print("MIP/Greedy objective:", obj_mip)
print("QUBO (approx) cost objective:", total_cost(df_assign_q))
display(df_assign_q.head())



## Wrap-up

- **MIP** enforces hard constraints exactly (when solver available).  
- **QUBO** uses penalties and only eligible variables; coverage and one-per-day are encouraged via squared/pairwise terms.  
- Preference handling converts utility into a dollar-equivalent to subtract from cost.

Try changing the parameters at the top (e.g., `SEATS_PER_SHIFT`, `MAX_SEATS_PER_EMP`, `PREF_BENEFIT`) and rerun.
